In [1]:
import pandas as pd 
import seaborn as sn 
import matplotlib.pyplot as plt 
import numpy as np 
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

### Loading Dataset 

In [2]:
df_full_feature = pd.read_csv('../Dataset/Normalisasi/processed_dataset_10_norm.csv',sep=',')
df_chi_square = pd.read_csv('../Dataset/Normalisasi/processed_dataset_chi_square_norm.csv', sep=',')
df_feature_selection = pd.read_csv('../Dataset/Normalisasi/processed_dataset_5_norm.csv',sep=',')
df_slice = pd.read_csv('../Dataset/Normalisasi/processed_dataset_slice_norm.csv',sep=',')

### Splitting Variable

In [3]:
# Dataset Full Feature
full_feature_x = df_full_feature[df_full_feature.columns[:9]]
full_feature_y = df_full_feature['label']

full_feature_x.head(5)

,src,dst,Protocol,pktrate,pktperflow,dur,dur_nsec,bytecount,pktcount
0,0.055556,0.647059,0.0,0.000000,0.001511,0.522594,0.235235,0.000640,0.003696
1,0.333333,0.705882,0.5,0.799687,0.799583,0.204147,0.114114,0.081254,0.851453
2,0.555556,0.882353,0.0,0.001565,0.001563,0.436470,0.243243,0.000534,0.003081
3,0.833333,0.941176,0.5,0.516432,0.516884,0.052100,0.622623,0.014585,0.125043
4,0.111111,0.647059,1.0,0.428795,0.428504,0.152578,0.579580,0.618070,0.335646


In [4]:
# Dataset Chi Square 
chi_square_x = df_chi_square[df_chi_square.columns[:5]]
chi_square_y = df_chi_square['label']

chi_square_x.head(5)

,dst,src,Protocol,pktcount,pktcount.1
0,0.647059,0.055556,0.0,0.003696,0.003696
1,0.705882,0.333333,0.5,0.851453,0.851453
2,0.882353,0.555556,0.0,0.003081,0.003081
3,0.941176,0.833333,0.5,0.125043,0.125043
4,0.647059,0.111111,1.0,0.335646,0.335646


In [5]:
# Dataset Feature Selection
feature_selection_x = df_feature_selection[df_feature_selection.columns[:5]]
feature_selection_y = df_feature_selection['label']

feature_selection_x.head(5)

,pktrate,pktperflow,Protocol,src,dst
0,0.000000,0.001511,0.0,0.055556,0.647059
1,0.799687,0.799583,0.5,0.333333,0.705882
2,0.001565,0.001563,0.0,0.555556,0.882353
3,0.516432,0.516884,0.5,0.833333,0.941176
4,0.428795,0.428504,1.0,0.111111,0.647059


In [6]:
# Dataset Slice
slice_x = df_slice[df_slice.columns[:4]]
slice_y= df_slice['label']

slice_x.head(5)

,dst,src,Protocol,bytecount
0,0.647059,0.055556,0.0,0.000640
1,0.705882,0.333333,0.5,0.081254
2,0.882353,0.555556,0.0,0.000534
3,0.941176,0.833333,0.5,0.014585
4,0.647059,0.111111,1.0,0.618070


### Full Model 

In [7]:
#Splitting Data 

x_train,x_test,y_train,y_test = train_test_split(full_feature_x,full_feature_y,test_size=0.30,random_state=9)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape


(50540, 9, 1)

In [8]:
model_full = Sequential()
model_full.add(LSTM(100, input_shape=(x_train.shape[1],1), return_sequences=True))
model_full.add(Dropout(0.5))
model_full.add(LSTM(32, return_sequences=False))
model_full.add(Dense(10, activation='relu'))
model_full.add(Dense(3, activation='softmax'))

model_full.compile(optimizer="adam",loss="sparse_categorical_crossentropy", metrics=['accuracy','MeanSquaredError', 'MeanAbsoluteError'])
model_full.fit(x_train, y_train, epochs=10)


Epoch 1/10
1580/1580 [==============================] - 20s 11ms/step - loss: 0.5913 - accuracy: 0.6787 - mean_squared_error: 0.3256 - mean_absolute_error: 0.4621
Epoch 2/10
1580/1580 [==============================] - 15s 10ms/step - loss: 0.2209 - accuracy: 0.9053 - mean_squared_error: 0.4158 - mean_absolute_error: 0.4637
Epoch 3/10
1580/1580 [==============================] - 18s 12ms/step - loss: 0.1808 - accuracy: 0.9193 - mean_squared_error: 0.4264 - mean_absolute_error: 0.4631
Epoch 4/10
1580/1580 [==============================] - 16s 10ms/step - loss: 0.1723 - accuracy: 0.9223 - mean_squared_error: 0.4281 - mean_absolute_error: 0.4635
Epoch 5/10
1580/1580 [==============================] - 17s 11ms/step - loss: 0.1660 - accuracy: 0.9248 - mean_squared_error: 0.4298 - mean_absolute_error: 0.4635 3s - loss: 0.1660 - accuracy: 0.9248 - 
Epoch 6/10
1580/1580 [==============================] - 18s 11ms/step - loss: 0.1615 - accuracy: 0.9269 - mean_squared_error: 0.4317 - mean_absol

In [9]:
y_pred_train = model_full.predict(x_train)
y_pred_test = model_full.predict(x_test)

In [10]:
import warnings
warnings.filterwarnings('ignore')
y_pred_train = [np.argmax(element) for element in y_pred_train]
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)


[[29009  1780]
 [ 1129 18622]]
              precision    recall  f1-score   support

           0       0.96      0.94      0.95     30789
           1       0.91      0.94      0.93     19751

    accuracy                           0.94     50540
   macro avg       0.94      0.94      0.94     50540
weighted avg       0.94      0.94      0.94     50540

Accuracy: 94.24416303917688


In [11]:
warnings.filterwarnings('ignore')
y_pred_test = [np.argmax(element) for element in y_pred_test]
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print("Accuracy:",accuracy_score(y_test, y_pred_test)*100)

[[12441   789]
 [  489  7942]]
              precision    recall  f1-score   support

           0       0.96      0.94      0.95     13230
           1       0.91      0.94      0.93      8431

    accuracy                           0.94     21661
   macro avg       0.94      0.94      0.94     21661
weighted avg       0.94      0.94      0.94     21661

Accuracy: 94.09999538340796


In [19]:
# Save Model To Pickle

model_full.save('lstm_full.h5')

Belajar Lagi Mengenai Komponen LSTM dan Kemudian Mengenai Konsep Reshaping Dari LSTM

## Model Chi

In [12]:
#Splitting Data 

x_train,x_test,y_train,y_test = train_test_split(chi_square_x,chi_square_y,test_size=0.30,random_state=9)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape

(50540, 5, 1)

In [14]:
model_chi = Sequential()
model_chi.add(LSTM(100, input_shape=(x_train.shape[1],1), return_sequences=True))
model_chi.add(Dropout(0.5))
model_chi.add(LSTM(32, return_sequences=False))
model_chi.add(Dense(10, activation='relu'))
model_chi.add(Dense(3, activation='softmax'))

model_chi.compile(optimizer="adam",loss="sparse_categorical_crossentropy", metrics=['accuracy','MeanSquaredError', 'MeanAbsoluteError'])
model_chi.fit(x_train, y_train, epochs=10)


Epoch 1/10
1580/1580 [==============================] - 16s 9ms/step - loss: 0.6459 - accuracy: 0.6257 - mean_squared_error: 0.3133 - mean_absolute_error: 0.4628
Epoch 2/10
1580/1580 [==============================] - 14s 9ms/step - loss: 0.4755 - accuracy: 0.6886 - mean_squared_error: 0.3523 - mean_absolute_error: 0.4633: 
Epoch 3/10
1580/1580 [==============================] - 13s 8ms/step - loss: 0.4280 - accuracy: 0.7434 - mean_squared_error: 0.3623 - mean_absolute_error: 0.4622
Epoch 4/10
1580/1580 [==============================] - 12s 8ms/step - loss: 0.4113 - accuracy: 0.7594 - mean_squared_error: 0.3687 - mean_absolute_error: 0.4646
Epoch 5/10
1580/1580 [==============================] - 12s 8ms/step - loss: 0.3956 - accuracy: 0.7669 - mean_squared_error: 0.3723 - mean_absolute_error: 0.4641
Epoch 6/10
1580/1580 [==============================] - 12s 7ms/step - loss: 0.3767 - accuracy: 0.7775 - mean_squared_error: 0.3775 - mean_absolute_error: 0.4642
Epoch 7/10
1580/1580 [====

In [15]:
y_pred_train = model_chi.predict(x_train)
y_pred_test = model_chi.predict(x_test)

In [16]:
import warnings
warnings.filterwarnings('ignore')
y_pred_train = [np.argmax(element) for element in y_pred_train]
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)


[[27525  3264]
 [ 6452 13299]]
              precision    recall  f1-score   support

           0       0.81      0.89      0.85     30789
           1       0.80      0.67      0.73     19751

    accuracy                           0.81     50540
   macro avg       0.81      0.78      0.79     50540
weighted avg       0.81      0.81      0.80     50540

Accuracy: 80.77562326869806


In [17]:
warnings.filterwarnings('ignore')
y_pred_test = [np.argmax(element) for element in y_pred_test]
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print("Accuracy:",accuracy_score(y_test, y_pred_test)*100)

[[11783  1447]
 [ 2799  5632]]
              precision    recall  f1-score   support

           0       0.81      0.89      0.85     13230
           1       0.80      0.67      0.73      8431

    accuracy                           0.80     21661
   macro avg       0.80      0.78      0.79     21661
weighted avg       0.80      0.80      0.80     21661

Accuracy: 80.39795023313789


In [20]:
model_chi.save('lstm_chi.h5')

## Feature Selection

In [21]:
#Splitting Data 

x_train,x_test,y_train,y_test = train_test_split(feature_selection_x,feature_selection_y,test_size=0.30,random_state=9)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape

(50540, 5, 1)

In [22]:
model_feature = Sequential()
model_feature.add(LSTM(100, input_shape=(x_train.shape[1],1), return_sequences=True))
model_feature.add(Dropout(0.5))
model_feature.add(LSTM(32, return_sequences=False))
model_feature.add(Dense(10, activation='relu'))
model_feature.add(Dense(3, activation='softmax'))

model_feature.compile(optimizer="adam",loss="sparse_categorical_crossentropy", metrics=['accuracy','MeanSquaredError', 'MeanAbsoluteError'])
model_feature.fit(x_train, y_train, epochs=10)


Epoch 1/10
1580/1580 [==============================] - 10s 5ms/step - loss: 0.5467 - accuracy: 0.7397 - mean_squared_error: 0.3395 - mean_absolute_error: 0.4627
Epoch 2/10
1580/1580 [==============================] - 11s 7ms/step - loss: 0.3694 - accuracy: 0.8549 - mean_squared_error: 0.3884 - mean_absolute_error: 0.4641
Epoch 3/10
1580/1580 [==============================] - 12s 7ms/step - loss: 0.3277 - accuracy: 0.8780 - mean_squared_error: 0.3988 - mean_absolute_error: 0.4636
Epoch 4/10
1580/1580 [==============================] - 12s 7ms/step - loss: 0.3132 - accuracy: 0.8881 - mean_squared_error: 0.4037 - mean_absolute_error: 0.4647
Epoch 5/10
1580/1580 [==============================] - 12s 7ms/step - loss: 0.2505 - accuracy: 0.9071 - mean_squared_error: 0.4122 - mean_absolute_error: 0.4620
Epoch 6/10
1580/1580 [==============================] - 13s 8ms/step - loss: 0.2373 - accuracy: 0.9077 - mean_squared_error: 0.4148 - mean_absolute_error: 0.4617: 1s - loss: 0.2374 - accurac

In [24]:
y_pred_train = model_feature.predict(x_train)
y_pred_test = model_feature.predict(x_test)

In [25]:
import warnings
warnings.filterwarnings('ignore')
y_pred_train = [np.argmax(element) for element in y_pred_train]
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)


[[27964  2825]
 [ 1828 17923]]
              precision    recall  f1-score   support

           0       0.94      0.91      0.92     30789
           1       0.86      0.91      0.89     19751

    accuracy                           0.91     50540
   macro avg       0.90      0.91      0.90     50540
weighted avg       0.91      0.91      0.91     50540

Accuracy: 90.79343094578552


In [26]:
warnings.filterwarnings('ignore')
y_pred_test = [np.argmax(element) for element in y_pred_test]
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print("Accuracy:",accuracy_score(y_test, y_pred_test)*100)

[[11997  1233]
 [  762  7669]]
              precision    recall  f1-score   support

           0       0.94      0.91      0.92     13230
           1       0.86      0.91      0.88      8431

    accuracy                           0.91     21661
   macro avg       0.90      0.91      0.90     21661
weighted avg       0.91      0.91      0.91     21661

Accuracy: 90.7898988966345


In [27]:
model_feature.save('lstm_feature.h5')

## Slice Model

In [7]:
#Splitting Data 

x_train,x_test,y_train,y_test = train_test_split(slice_x,slice_y,test_size=0.30,random_state=9)

x_train = x_train.to_numpy()
x_test = x_test.to_numpy()
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
x_train.shape

(50540, 4, 1)

In [9]:
model_slice = Sequential()
model_slice.add(LSTM(100, input_shape=(x_train.shape[1],1), return_sequences=True))
model_slice.add(Dropout(0.5))
model_slice.add(LSTM(32, return_sequences=False))
model_slice.add(Dense(10, activation='relu'))
model_slice.add(Dense(2, activation='softmax'))
#model_slice.add(Dense(3, activation='softmax'))

model_slice.compile(optimizer="adam",loss="sparse_categorical_crossentropy", metrics=['accuracy','MeanSquaredError', 'MeanAbsoluteError'])
model_slice.fit(x_train, y_train, epochs=50)


Epoch 1/50
1580/1580 [==============================] - 19s 8ms/step - loss: 0.5823 - accuracy: 0.6458 - mean_squared_error: 0.2913 - mean_absolute_error: 0.5000
Epoch 2/50
1580/1580 [==============================] - 13s 8ms/step - loss: 0.4326 - accuracy: 0.7413 - mean_squared_error: 0.3456 - mean_absolute_error: 0.5000
Epoch 3/50
1580/1580 [==============================] - 13s 8ms/step - loss: 0.4210 - accuracy: 0.7507 - mean_squared_error: 0.3502 - mean_absolute_error: 0.5000
Epoch 4/50
1580/1580 [==============================] - 12s 8ms/step - loss: 0.4123 - accuracy: 0.7571 - mean_squared_error: 0.3537 - mean_absolute_error: 0.5000
Epoch 5/50
1580/1580 [==============================] - 13s 8ms/step - loss: 0.4081 - accuracy: 0.7607 - mean_squared_error: 0.3555 - mean_absolute_error: 0.5000: 2s - loss: 0.4079 - accuracy: 0.7608 - mean_squared_e
Epoch 6/50
1580/1580 [==============================] - 12s 8ms/step - loss: 0.4074 - accuracy: 0.7609 - mean_squared_error: 0.3555 - m

In [10]:
y_pred_train = model_slice.predict(x_train)
y_pred_test = model_slice.predict(x_test)

In [11]:
import warnings
warnings.filterwarnings('ignore')
y_pred_train = [np.argmax(element) for element in y_pred_train]
print(confusion_matrix(y_train, y_pred_train))
print(classification_report(y_train, y_pred_train))
print("Accuracy:",accuracy_score(y_train, y_pred_train)*100)


[[27068  3721]
 [ 2054 17697]]
              precision    recall  f1-score   support

           0       0.93      0.88      0.90     30789
           1       0.83      0.90      0.86     19751

    accuracy                           0.89     50540
   macro avg       0.88      0.89      0.88     50540
weighted avg       0.89      0.89      0.89     50540

Accuracy: 88.57340720221607


In [12]:
warnings.filterwarnings('ignore')
y_pred_test = [np.argmax(element) for element in y_pred_test]
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))
print("Accuracy:",accuracy_score(y_test, y_pred_test)*100)

[[11534  1696]
 [  932  7499]]
              precision    recall  f1-score   support

           0       0.93      0.87      0.90     13230
           1       0.82      0.89      0.85      8431

    accuracy                           0.88     21661
   macro avg       0.87      0.88      0.87     21661
weighted avg       0.88      0.88      0.88     21661

Accuracy: 87.86759614052906


In [13]:
model_slice.save('lstm_slice.h5')